<a href="https://colab.research.google.com/github/darrenpierre90/Machine-Learning-Sentiment-Analysis/blob/master/COMP4900A2Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
drive.mount('/content/drive')
TRAIN_DATA="/content/drive/My Drive/Colab Notebooks/Data Sets/train.csv"
TEST_DATA="/content/drive/My Drive/Colab Notebooks/Data Sets/test.csv"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Loading up datasets into dataframes
import pandas as pd 
import numpy as np 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
trainDS=pd.read_csv(TRAIN_DATA)
testDS=pd.read_csv(TEST_DATA)


In [0]:
# Feature engineering test
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(trainDS.review, trainDS.sentiment, train_size=0.80, test_size=0.20)
set_of_vectorizers=[CountVectorizer(max_features=5000),
                   TfidfVectorizer(max_features=5000),
                   CountVectorizer(max_features=5000,ngram_range=(1,2))]
                                   
names=["CountVectorizer","TfidfVectorizer","N-gram CountVectorizer"]


for i in range(len(set_of_vectorizers)):
  X=set_of_vectorizers[i].fit_transform(X_train)
  clf = LogisticRegression(max_iter=90000, C=1).fit(X,y_train)
  X_test_v=set_of_vectorizers[i].transform(X_test)
  pred_y=clf.predict(X_test_v)
  print(f"{names[i]} accuracy score {accuracy_score(pred_y,y_test)}")





CountVectorizer accuracy score 0.8435
TfidfVectorizer accuracy score 0.8613333333333333
N-gram CountVectorizer accuracy score 0.8375


In [0]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import 	WordNetLemmatizer
import re

def removeStopwords(sentence):
  stop_words = set(stopwords.words('english')) 
  filtered_sentence = [w for w in word_tokenize(sentence)  if not w in stop_words] 
  return " ".join(filtered_sentence)

def remove_punctuation(sentence):
  return sentence.translate(str.maketrans('', '', string.punctuation))

def remove_html_tags(sentence):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', sentence)
  return cleantext

# Normalization of words

def stemmize(sentence):
  stemmer = SnowballStemmer("english")
  stemmize_tokens=[stemmer.stem(token) for token in word_tokenize(sentence)]
  return " ".join(stemmize_tokens)

def lemmatize(sentence):
  wordnet = WordNetLemmatizer()
  lem_words=[wordnet.lemmatize(w) for w in word_tokenize(sentence)]
  return " ".join(lem_words)


def baseCleaner(sentence):
  func_cleaners=[removeStopwords,remove_html_tags,remove_punctuation]
  sentence=sentence
  for clean_func in func_cleaners:
    sentence=clean_func(sentence)
  return sentence
# normalize and use base cleaners to clean text
def stemBC(sentence):
  return(stemmize(baseCleaner(sentence)))
def lemBC(sentence):
  return(lemmatize(baseCleaner(sentence)))

# use both normalization techiniuqes considering the order  
def both1(sentence):
  return(stemmize(lemBC(sentence)))
def both2(sentence):
  return(lemmatize(stemBC(sentence)))



In [0]:
# create datasets with clean data
stemmize_DS=trainDS.review.apply(stemBC)
lemmized_DS=trainDS.review.apply(lemBC)
both1_DS=trainDS.review.apply(both1)
both2_DS=trainDS.review.apply(both2)

In [0]:
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import cross_val_score

kf = KFold(n_splits=10)
vectorizer = TfidfVectorizer(max_features=5000)
experimentDS=[trainDS.review,stemmize_DS,lemmized_DS,both1_DS,both2_DS]

for ds in experimentDS:
  vectors_train = vectorizer.fit_transform(ds)
  logReg=LogisticRegression(max_iter=90000000)
  scores = cross_val_score(logReg, vectors_train, trainDS.sentiment, cv=kf)
  avg_score = np.mean(scores)
  print(avg_score)

0.8668666666666667
0.8678000000000001
0.8648666666666667
0.8678666666666667
0.8676999999999999


In [0]:
'''
# forloop iteration testing
from sklearn.metrics import classification_report


max = -1
index = -1
for i in range(0, 0):
  vectorizer = TfidfVectorizer(max_features=50000)
  vectors_train = vectorizer.fit_transform(X_train)
  clf = LogisticRegression(max_iter=90000000).fit(vectors_train, y_train)
  vectors_test=vectorizer.transform(X_test)
  predicted_values=clf.predict(vectors_test)
  print("***INDEX***", i)
  print(classification_report(predicted_values,y_test))

'''

In [0]:
#from sklearn.metrics import classification_report
vectors_test=vectorizer.transform(X_test)
predicted_values=clf.predict(vectors_test)
print(classification_report(predicted_values,y_test))

vectorRealTest=vectorizer.transform(testDS.review)
values=clf.predict(vectorRealTest)
df = pd.DataFrame()
df["id"]=testDS.id
df["answer"]=values

print(df)

from google.colab import files
df.to_csv('filename.csv') 
#files.download('filename.csv')
 

In [0]:
  """
  # Title: Bernoulli Naive Bayes Classifier
  # Author:  Robert M. Johnson 
  # Date: <date>
  # Code version: 1
  # Availability: https://mattshomepage.com/articles/2016/Jun/07/bernoulli_nb/
  """
def get_features(text):
    return set([w.lower() for w in text.split(" ")])

import numpy as np
from math import log

from collections import Counter
class NaiveBayesB():
    def __init__(self):
      self._log_priors = None
      self._cond_probs = None
      self.features = None

    def fit(self, documents, labels):

        label_counts = Counter(labels)
        N = float(sum(label_counts.values()))
        self._log_priors = {k: log(v/N) for k, v in label_counts.items()}

        X = [set(get_features(d)) for d in documents]
        self.features = set([f for features in X for f in features])
        self._cond_probs = {l: {f: 0. for f in self.features} for l in self._log_priors}

        for x, l in zip(X, labels):
            for f in x:
                self._cond_probs[l][f] += 1.

        for l in self._cond_probs:
            N = label_counts[l]
            self._cond_probs[l] = {f: (v + 1.) / (N + 2.) for f, v in self._cond_probs[l].items()}

    def predict(self, text):
      x = get_features(text)
      pred_class = None
      max_ = float("-inf")

      for l in self._log_priors:
          log_sum = self._log_priors[l]
          for f in self.features:
              prob = self._cond_probs[l][f]
              log_sum += log(prob if f in x else 1. - prob)
          if log_sum > max_:
              max_ = log_sum
              pred_class = l

      return pred_class

    def predictDS(self,dataset):
      return [self.predict(instance) for instance in dataset]


In [0]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# Loggistic Regression section
NUM_FOLDS=10
kf = KFold(n_splits=NUM_FOLDS)
X=both1_DS
y=trainDS.sentiment
vectorizer = TfidfVectorizer(max_features=5000).fit(X)
logRegression= LogisticRegression(max_iter=90000, C=1)
scores = cross_val_score(logRegression, vectorizer.fit_transform(X), y, cv=kf)
avg_score = np.mean(scores)
print(f"Logistic Regression result: {avg_score}")

# naive Bayes
scores=list()
for train_index, test_index in kf.split(X,y):
  clf=NaiveBayesB()
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  # Naive Bayes Model
  clf.fit(X_train,y_train)

  predictions=clf.predictDS(X_test)

  scores.append(accuracy_score(y_test,predictions))

result=np.mean(scores)
print(f"Naive Bayes Result:{result}")



0.8635666666666667
0.8464333333333333


In [0]:

ds=[trainDS.review]

for x in ds:
  cls=LogisticRegression(max_iter=90000)
  vectorizer=TfidfVectorizer(max_features=50000)
  X=vectorizer.fit_transform(x)
  cls.fit(X,trainDS.sentiment)
  vectorRealTest=vectorizer.transform(testDS.review)
  values=cls.predict(vectorRealTest)
  df = pd.DataFrame()
  df["sentiment"]=values

  from google.colab import files
  df.to_csv('filename.csv') 
  files.download('filename.csv')
  
  
